In [ ]:
casualties = read.csv('Casualties.csv')
crash = read.csv('Crash.csv')
datatime = read.csv('DateTime.csv')
description = read.csv('Description.csv')
location = read.csv('Location.csv')
vehicles = read.csv('Vehicles.csv')

In [ ]:
require(data.table)
setDT(casualties); setDT(crash) # convert to data.tables by reference
z1 = casualties[crash, mult = "first", on = "casualties_id", nomatch=0L]

require(data.table)
setDT(datatime); setDT(z1) # convert to data.tables by reference
z2 = datatime[z1, mult = "first", on = "date_time_id", nomatch=0L]

require(data.table)
setDT(description); setDT(z2) # convert to data.tables by reference
z3 = description[z2, mult = "first", on = "description_id", nomatch=0L]

require(data.table)
setDT(location); setDT(z3) # convert to data.tables by reference
z4 = location[z3, mult = "first", on = "lat_long", nomatch=0L]

require(data.table)
setDT(vehicles); setDT(z4) # convert to data.tables by reference
z5 = vehicles[z4, mult = "first", on = "vehicles_id", nomatch=0L]

# Change speed limit to number   
# WA and ACT not include speed_limit
z5[z5$speed_limit=='0 - 50 km/h',]$speed_limit = 50
z5[z5$speed_limit=='60 km/h',]$speed_limit=60
z5[z5$speed_limit=='70 km/h',]$speed_limit=70
z5[z5$speed_limit=='80 - 90 km/h',]$speed_limit=90
z5[z5$speed_limit=='100 - 110 km/h',]$speed_limit=110
z5[z5$state == 'NZ',]$speed_limit = as.integer(z5[z5$state == 'NZ',]$speed_limit)
# change chr to int
z5[,'speed_limit'] = as.integer(z5[,'speed_limit'])

# change severity to number damage levels
#property_damage = 1
#minor_injury = 2
#serious_injury = 3
#fatality = 4 
z5[z5$severity=='property_damage',]$severity = 1
z5[z5$severity=='minor_injury',]$severity = 2
z5[z5$severity=='serious_injury',]$severity = 3
z5[z5$severity=='fatality',]$severity = 4
z5$severity = as.integer(z5$severity)

# midblock T/F to 1/0
z5[z5$midblock == 'True',]$midblock = 1
z5[z5$midblock == 'False',]$midblock = 0

# intersection 
z5[z5$intersection== 'True',]$intersection = 1
z5[z5$intersection == 'False',]$intersection = 0

# road_sealed
z5[z5$road_sealed== 'True',]$road_sealed = 1
z5[z5$road_sealed == 'False',]$road_sealed = 0

# road_wet
z5[z5$road_wet== 'True',]$road_wet = 1
z5[z5$road_wet == 'False',]$road_wet = 0

In [ ]:
# write to a csv file
# write.csv(z5,'merge_dataset.csv')

# read the file 
# x = read.csv('merge_dataset.csv')

In [ ]:
colnames(x)

In [ ]:
# for each state
sa = x[x$state == 'SA',]
qld = x[x$state == 'QLD',]
vic = x[x$state == 'VIC',]
wa = x[x$state == 'WA',]
act = x[x$state == 'ACT',]
nz = x[x$state == 'NZ',]

In [ ]:
# 4 states data
td = rbind(sa,qld,vic,nz)

In [ ]:
###################

xgboost model

###################

In [ ]:
library(Matrix)
library(xgboost)

list1 = c('severity', 'speed_limit','day_of_week','hour','midblock',
          'intersection','road_sealed','road_wet','animals','car_sedan'
          ,'car_utility','car_van','car_4x4','car_station_wagon','motor_cycle','truck_small',
         'truck_large','bus','taxi','bicycle','scooter','pedestrian','inanimate','train','tram',
         'vehicle_other')

In [ ]:
rows = round(nrow(td)*0.7)
train <- td[1:rows,list1]
test <- td[rows:nrow(td),list1]

traindata1 <- data.matrix(train[,2:26])
traindata2 <- Matrix(traindata1,sparse = T)
traindata3 <- train[,1]
traindata4 <- list(data = traindata2,label = traindata3)

testdata1 <- data.matrix(test[,2:26])
testdata2 <- Matrix(testdata1,sparse = T)
testdata3 <- test[,1]
testdata4 <- list(data = testdata2,label = testdata3)

dtrain <- xgb.DMatrix(data = traindata4$data,label = traindata4$label)
dtest <- xgb.DMatrix(data = testdata4$data,label = testdata4$label)

In [ ]:
bst = xgboost(data =dtrain,eta = 0.6,max_depth = 30, nrounds = 30,
             objective = "reg:linear")

In [ ]:
pre = round(predict(bst,dtest))
table(pre)